In [55]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/transacciones_logistica_v2.csv')

In [56]:
df.columns

Index(['Transaccion_ID', 'SKU_ID', 'Fecha_Venta', 'Cantidad_Vendida',
       'Precio_Venta_Final', 'Costo_Envio', 'Tiempo_Entrega_Real',
       'Estado_Envio', 'Ciudad_Destino', 'Canal_Venta'],
      dtype='object')

In [57]:
Transaccion_ID = 'Transaccion_ID'
SKUid = 'SKU_ID'
Fecha_venta = 'Fecha_Venta'
Cantidad_vendida = 'Cantidad_Vendida'
Precio_venta_final = 'Precio_Venta_Final'
Costo_envio = 'Costo_Envio'
Tiempo_entrega_real = 'Tiempo_Entrega_Real'
Estado_envio = 'Estado_Envio'
Ciudad_destino = 'Ciudad_Destino'
Canal_venta = 'Canal_Venta'


In [58]:
df[Transaccion_ID].describe()

count         10000
unique        10000
top       TRX-10000
freq              1
Name: Transaccion_ID, dtype: object

In [59]:
df[Transaccion_ID].isna().sum()
df[Transaccion_ID].isnull().sum()

np.int64(0)

In [60]:
df[SKUid].describe()

count         10000
unique         2889
top       PROD-1229
freq             11
Name: SKU_ID, dtype: object

In [61]:
df[SKUid].isnull().sum()


np.int64(0)

In [62]:
df[Tiempo_entrega_real].describe()

count    10000.000000
mean        19.881900
std         69.914879
min          1.000000
25%          8.000000
50%         15.000000
75%         22.000000
max        999.000000
Name: Tiempo_Entrega_Real, dtype: float64

In [63]:
df[Cantidad_vendida].describe()

count    10000.000000
mean         7.413600
std          4.205485
min         -5.000000
25%          4.000000
50%          7.000000
75%         11.000000
max         14.000000
Name: Cantidad_Vendida, dtype: float64

In [64]:
df[Costo_envio].describe()

count    9166.000000
mean       52.411765
std        27.419908
min         5.000000
25%        28.802500
50%        52.360000
75%        76.110000
max        99.990000
Name: Costo_Envio, dtype: float64

In [65]:
df[Costo_envio].isna().sum()

np.int64(834)

In [66]:
df[Precio_venta_final].describe()

count    10000.000000
mean       996.392443
std        570.571003
min         10.010000
25%        500.265000
50%        999.230000
75%       1479.225000
max       1999.580000
Name: Precio_Venta_Final, dtype: float64

In [67]:
print(df[Ciudad_destino].describe())
print(df[Ciudad_destino].isna().sum())
print(df[Ciudad_destino].value_counts())


count          10000
unique             8
top       Ventas_Web
freq            1290
Name: Ciudad_Destino, dtype: object
0
Ciudad_Destino
Ventas_Web      1290
BOG             1267
Bogotá          1261
Cali            1256
Bucaramanga     1250
Medellín        1234
MED             1223
Barranquilla    1219
Name: count, dtype: int64


In [68]:
# Reemplazamos las abreviaturas de ciudades por nombres completos
df[Ciudad_destino] = df[Ciudad_destino].replace({'BOG': 'Bogotá', 'MED': 'Medellín'})
print("Valores después del reemplazo:")
print(df[Ciudad_destino].value_counts())

Valores después del reemplazo:
Ciudad_Destino
Bogotá          2528
Medellín        2457
Ventas_Web      1290
Cali            1256
Bucaramanga     1250
Barranquilla    1219
Name: count, dtype: int64


In [69]:
df[Canal_venta].describe()
df[Canal_venta].value_counts()
df['Canal_Venta'] = df['Canal_Venta'].replace({ # Consideramos que es importante manteneer App como un canal dee venta debido a que nos puede dar informacion relevante con el uso de la aplicacion y la necesidad de mantenerla.
        'WhatsApp': 'Online',
    })

In [70]:
df.columns
df[Fecha_venta].describe()

count          10000
unique           500
top       12/01/2025
freq              39
Name: Fecha_Venta, dtype: object

In [71]:
df[Fecha_venta] = pd.to_datetime(df[Fecha_venta], format="%d/%m/%Y", errors="coerce")
df[Fecha_venta].describe()

count                         10000
mean     2025-05-29 15:39:18.720000
min             2024-09-23 00:00:00
25%             2025-01-24 00:00:00
50%             2025-05-28 00:00:00
75%             2025-10-02 00:00:00
max             2026-02-04 00:00:00
Name: Fecha_Venta, dtype: object

In [72]:
df[Cantidad_vendida].describe()
df[Cantidad_vendida] = df[Cantidad_vendida].abs() # Encontramos valores negativos en cantidad vendida, los cuales no tienen sentido en este contexto, por lo que tomamos su valor absoluto.
df[Cantidad_vendida].describe()

count    10000.000000
mean         7.513600
std          4.024082
min          1.000000
25%          4.000000
50%          7.000000
75%         11.000000
max         14.000000
Name: Cantidad_Vendida, dtype: float64

In [73]:
df.columns
df[Precio_venta_final].describe()

count    10000.000000
mean       996.392443
std        570.571003
min         10.010000
25%        500.265000
50%        999.230000
75%       1479.225000
max       1999.580000
Name: Precio_Venta_Final, dtype: float64

In [74]:
df[Tiempo_entrega_real].describe()


count    10000.000000
mean        19.881900
std         69.914879
min          1.000000
25%          8.000000
50%         15.000000
75%         22.000000
max        999.000000
Name: Tiempo_Entrega_Real, dtype: float64

## Tratamiento de Outliers con Cuantiles

Aplicamos el método IQR (Interquartile Range) para detectar y reemplazar outliers en `Tiempo_Entrega_Real`:
- Calculamos Q1 (percentil 25%) y Q3 (percentil 75%)
- IQR = Q3 - Q1
- Límite inferior = Q1 - 1.5 * IQR
- Límite superior = Q3 + 1.5 * IQR
- Valores fuera de estos límites son outliers

Probamos 4 métodos diferentes de reemplazo:

In [75]:
# Recargar datos originales para el análisis de outliers
df_original = pd.read_csv('data/transacciones_logistica_v2.csv')
print("Estadísticas de Tiempo_Entrega_Real (datos originales):")
print(df_original['Tiempo_Entrega_Real'].describe())

Estadísticas de Tiempo_Entrega_Real (datos originales):
count    10000.000000
mean        19.881900
std         69.914879
min          1.000000
25%          8.000000
50%         15.000000
75%         22.000000
max        999.000000
Name: Tiempo_Entrega_Real, dtype: float64


In [76]:
def reemplazar_outliers_tiempo_entrega_real(df, metodo):
    """
    Reemplaza outliers en Tiempo_Entrega_Real usando el método IQR.
    
    Parámetros:
    - df: DataFrame a procesar (se modifica directamente)
    - metodo: 'limite', 'media', 'mediana', 'moda'
    """
    Q1 = df['Tiempo_Entrega_Real'].quantile(0.25)
    Q3 = df['Tiempo_Entrega_Real'].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    # No puede haber tiempos negativos
    limite_inferior = max(limite_inferior, 0)
    
    mascara_outliers = (df['Tiempo_Entrega_Real'] < limite_inferior) | (df['Tiempo_Entrega_Real'] > limite_superior)
    
    # Aplicar el método de reemplazo
    if metodo == 'limite':
        df['Tiempo_Entrega_Real'] = df['Tiempo_Entrega_Real'].clip(lower=limite_inferior, upper=limite_superior)
    elif metodo == 'media':
        valor_reemplazo = df['Tiempo_Entrega_Real'][~mascara_outliers].mean()
        df.loc[mascara_outliers, 'Tiempo_Entrega_Real'] = valor_reemplazo
    elif metodo == 'mediana':
        valor_reemplazo = df['Tiempo_Entrega_Real'][~mascara_outliers].median()
        df.loc[mascara_outliers, 'Tiempo_Entrega_Real'] = valor_reemplazo
    elif metodo == 'moda':
        valor_reemplazo = df['Tiempo_Entrega_Real'][~mascara_outliers].mode()[0]
        df.loc[mascara_outliers, 'Tiempo_Entrega_Real'] = valor_reemplazo
    
    return df

### Opción 1: Reemplazo por límites (Capping/Winsorizing)

In [77]:
#df = reemplazar_outliers_tiempo_entrega_real(df, metodo='limite')

### Opción 2: Reemplazo por Media

In [78]:
#df = reemplazar_outliers_tiempo_entrega_real(df, metodo='media')

### Opción 3: Reemplazo por Mediana

In [79]:
df = reemplazar_outliers_tiempo_entrega_real(df, metodo='mediana')

### Opción 4: Reemplazo por Moda

In [80]:
#df = reemplazar_outliers_tiempo_entrega_real(df, metodo='moda')

In [81]:
df[Costo_envio].describe()

count    9166.000000
mean       52.411765
std        27.419908
min         5.000000
25%        28.802500
50%        52.360000
75%        76.110000
max        99.990000
Name: Costo_Envio, dtype: float64

In [82]:
df.describe()

,Fecha_Venta,Cantidad_Vendida,Precio_Venta_Final,Costo_Envio,Tiempo_Entrega_Real
count,10000,10000.000000,10000.000000,9166.000000,10000.00000
mean,2025-05-29 15:39:18.720000,7.513600,996.392443,52.411765,14.96190
min,2024-09-23 00:00:00,1.000000,10.010000,5.000000,1.00000
25%,2025-01-24 00:00:00,4.000000,500.265000,28.802500,8.00000
50%,2025-05-28 00:00:00,7.000000,999.230000,52.360000,15.00000
75%,2025-10-02 00:00:00,11.000000,1479.225000,76.110000,22.00000
max,2026-02-04 00:00:00,14.000000,1999.580000,99.990000,29.00000
std,NaN,4.024082,570.571003,27.419908,8.37616


In [83]:
def imputar_costo_envio(df, remplzar_por='Mediana'):
    """
    Imputa valores faltantes en Costo_Envio con la media.
    """
    if remplzar_por == 'Mediana':
        mediana_costo_envio = df['Costo_Envio'].median()
        df['Costo_Envio'] = df['Costo_Envio'].fillna(mediana_costo_envio)
    elif remplzar_por == 'Media':
        media_costo_envio = df['Costo_Envio'].mean()
        df['Costo_Envio'] = df['Costo_Envio'].fillna(media_costo_envio)
    elif remplzar_por == 'Moda':
        moda_costo_envio = df['Costo_Envio'].mode()[0]
        df['Costo_Envio'] = df['Costo_Envio'].fillna(moda_costo_envio)
    return df
df = imputar_costo_envio(df, remplzar_por='Mediana')

In [84]:
df.describe()

,Fecha_Venta,Cantidad_Vendida,Precio_Venta_Final,Costo_Envio,Tiempo_Entrega_Real
count,10000,10000.000000,10000.000000,10000.000000,10000.00000
mean,2025-05-29 15:39:18.720000,7.513600,996.392443,52.407448,14.96190
min,2024-09-23 00:00:00,1.000000,10.010000,5.000000,1.00000
25%,2025-01-24 00:00:00,4.000000,500.265000,30.700000,8.00000
50%,2025-05-28 00:00:00,7.000000,999.230000,52.360000,15.00000
75%,2025-10-02 00:00:00,11.000000,1479.225000,74.092500,22.00000
max,2026-02-04 00:00:00,14.000000,1999.580000,99.990000,29.00000
std,NaN,4.024082,570.571003,26.251493,8.37616


In [85]:
df.columns

Index(['Transaccion_ID', 'SKU_ID', 'Fecha_Venta', 'Cantidad_Vendida',
       'Precio_Venta_Final', 'Costo_Envio', 'Tiempo_Entrega_Real',
       'Estado_Envio', 'Ciudad_Destino', 'Canal_Venta'],
      dtype='object')

In [86]:
df[Estado_envio].describe()
df[Estado_envio].value_counts()

Estado_Envio
Retrasado    1757
Entregado    1684
Devuelto     1645
En Camino    1628
Perdido      1603
Name: count, dtype: int64

In [87]:
df_original.head(10)

,Transaccion_ID,SKU_ID,Fecha_Venta,Cantidad_Vendida,Precio_Venta_Final,Costo_Envio,Tiempo_Entrega_Real,Estado_Envio,Ciudad_Destino,Canal_Venta
0,TRX-10000,PROD-3619,25/04/2025,-5,1752.48,NaN,999,Entregado,Bucaramanga,Físico
1,TRX-10001,PROD-1456,29/07/2025,12,686.88,67.16,29,Entregado,Ventas_Web,WhatsApp
2,TRX-10002,PROD-1102,31/05/2025,2,402.06,32.55,13,Perdido,MED,Físico
3,TRX-10003,PROD-2126,23/01/2026,4,665.98,62.64,12,Perdido,Medellín,Online
4,TRX-10004,PROD-2003,09/10/2025,5,1222.62,52.91,15,Entregado,BOG,App
5,TRX-10005,PROD-1914,05/11/2025,2,740.61,25.63,29,Retrasado,Medellín,Online
6,TRX-10006,PROD-1571,20/06/2025,12,1349.06,30.32,23,NaN,Bogotá,Físico
7,TRX-10007,PROD-1419,18/11/2025,8,481.50,21.21,13,Perdido,Cali,App
8,TRX-10008,PROD-3771,03/11/2025,1,1404.98,10.38,6,NaN,Medellín,App
9,TRX-10009,PROD-3233,21/03/2025,3,701.40,63.82,26,NaN,MED,Físico


In [88]:
df.head(10)

,Transaccion_ID,SKU_ID,Fecha_Venta,Cantidad_Vendida,Precio_Venta_Final,Costo_Envio,Tiempo_Entrega_Real,Estado_Envio,Ciudad_Destino,Canal_Venta
0,TRX-10000,PROD-3619,2025-04-25,5,1752.48,52.36,15,Entregado,Bucaramanga,Físico
1,TRX-10001,PROD-1456,2025-07-29,12,686.88,67.16,29,Entregado,Ventas_Web,Online
2,TRX-10002,PROD-1102,2025-05-31,2,402.06,32.55,13,Perdido,Medellín,Físico
3,TRX-10003,PROD-2126,2026-01-23,4,665.98,62.64,12,Perdido,Medellín,Online
4,TRX-10004,PROD-2003,2025-10-09,5,1222.62,52.91,15,Entregado,Bogotá,App
5,TRX-10005,PROD-1914,2025-11-05,2,740.61,25.63,29,Retrasado,Medellín,Online
6,TRX-10006,PROD-1571,2025-06-20,12,1349.06,30.32,23,NaN,Bogotá,Físico
7,TRX-10007,PROD-1419,2025-11-18,8,481.50,21.21,13,Perdido,Cali,App
8,TRX-10008,PROD-3771,2025-11-03,1,1404.98,10.38,6,NaN,Medellín,App
9,TRX-10009,PROD-3233,2025-03-21,3,701.40,63.82,26,NaN,Medellín,Físico


In [89]:
## Análisis de Estado_Envio para Imputación

#Analizamos la relación entre Tiempo_Entrega_Real y Estado_Envio para imputar valores nulos basados en el tiempo de entrega.

In [90]:
# Verificar valores nulos en Estado_Envio
print("Valores nulos en Estado_Envio:")
print(df[Estado_envio].isna().sum())
print(f"\nTotal de registros: {len(df)}")
print(f"Porcentaje de nulos: {df[Estado_envio].isna().sum() / len(df) * 100:.2f}%")

Valores nulos en Estado_Envio:
1683

Total de registros: 10000
Porcentaje de nulos: 16.83%


In [91]:
# Analizar los valores únicos de Estado_Envio
print("Valores únicos en Estado_Envio:")
print(df[Estado_envio].value_counts(dropna=False))
print("\n" + "="*60)

Valores únicos en Estado_Envio:
Estado_Envio
Retrasado    1757
Entregado    1684
NaN          1683
Devuelto     1645
En Camino    1628
Perdido      1603
Name: count, dtype: int64



In [92]:
# Analizar la relación entre Tiempo_Entrega_Real y Estado_Envio
# Estadísticas de tiempo de entrega por cada estado
print("Estadísticas de Tiempo_Entrega_Real por Estado_Envio:\n")

for estado in df[Estado_envio].dropna().unique():
    datos_estado = df[df[Estado_envio] == estado][Tiempo_entrega_real]
    print(f"Estado: {estado}")
    print(f"  Cantidad: {len(datos_estado)}")
    print(f"  Media: {datos_estado.mean():.2f} días")
    print(f"  Mediana: {datos_estado.median():.2f} días")
    print(f"  Min: {datos_estado.min():.2f} días")
    print(f"  Max: {datos_estado.max():.2f} días")
    print(f"  Q1 (25%): {datos_estado.quantile(0.25):.2f} días")
    print(f"  Q3 (75%): {datos_estado.quantile(0.75):.2f} días")
    print("-" * 50)

Estadísticas de Tiempo_Entrega_Real por Estado_Envio:

Estado: Entregado
  Cantidad: 1684
  Media: 14.94 días
  Mediana: 15.00 días
  Min: 1.00 días
  Max: 29.00 días
  Q1 (25%): 8.00 días
  Q3 (75%): 22.00 días
--------------------------------------------------
Estado: Perdido
  Cantidad: 1603
  Media: 14.87 días
  Mediana: 15.00 días
  Min: 1.00 días
  Max: 29.00 días
  Q1 (25%): 8.00 días
  Q3 (75%): 22.00 días
--------------------------------------------------
Estado: Retrasado
  Cantidad: 1757
  Media: 15.17 días
  Mediana: 15.00 días
  Min: 1.00 días
  Max: 29.00 días
  Q1 (25%): 8.00 días
  Q3 (75%): 23.00 días
--------------------------------------------------
Estado: Devuelto
  Cantidad: 1645
  Media: 14.79 días
  Mediana: 15.00 días
  Min: 1.00 días
  Max: 29.00 días
  Q1 (25%): 8.00 días
  Q3 (75%): 22.00 días
--------------------------------------------------
Estado: En Camino
  Cantidad: 1628
  Media: 14.98 días
  Mediana: 15.00 días
  Min: 1.00 días
  Max: 29.00 días
  Q1

In [93]:
# Ver distribución de tiempos de entrega por estado usando groupby
resumen_por_estado = df.groupby(Estado_envio)[Tiempo_entrega_real].agg([
    ('count', 'count'),
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max'),
    ('Q1', lambda x: x.quantile(0.25)),
    ('Q3', lambda x: x.quantile(0.75))
]).round(2)

print("\nResumen agrupado por Estado_Envio:")
print(resumen_por_estado)


Resumen agrupado por Estado_Envio:
              count   mean  median   std  min  max   Q1    Q3
Estado_Envio                                                 
Devuelto       1645  14.79    15.0  8.42    1   29  8.0  22.0
En Camino      1628  14.98    15.0  8.25    1   29  8.0  22.0
Entregado      1684  14.94    15.0  8.43    1   29  8.0  22.0
Perdido        1603  14.87    15.0  8.42    1   29  8.0  22.0
Retrasado      1757  15.17    15.0  8.49    1   29  8.0  23.0


In [94]:
# Analizar los registros con valores nulos en Estado_Envio
print("Registros con Estado_Envio nulo:")
registros_nulos = df[df[Estado_envio].isna()]
print(f"\nCantidad: {len(registros_nulos)}")
print(f"\nEstadísticas de Tiempo_Entrega_Real para registros con Estado_Envio nulo:")
print(registros_nulos[Tiempo_entrega_real].describe())

Registros con Estado_Envio nulo:

Cantidad: 1683

Estadísticas de Tiempo_Entrega_Real para registros con Estado_Envio nulo:
count    1683.000000
mean       15.008913
std         8.243502
min         1.000000
25%         8.000000
50%        15.000000
75%        22.000000
max        29.000000
Name: Tiempo_Entrega_Real, dtype: float64


### Conclusión del Análisis

Los tiempos de entrega son muy similares entre todos los estados (media ~15 días), lo que indica que **no hay una relación clara entre el tiempo de entrega y el estado**. 

Dado esto, la mejor estrategia de imputación es usar la **moda** (valor más frecuente) del Estado_Envio.

In [95]:
# Imputar valores nulos de Estado_Envio con la moda
moda_estado_envio = df[Estado_envio].mode()[0]
print(f"Valor más frecuente (moda) de Estado_Envio: {moda_estado_envio}")
print(f"\nNulos antes de imputación: {df[Estado_envio].isna().sum()}")

df[Estado_envio] = df[Estado_envio].fillna(moda_estado_envio)

print(f"Nulos después de imputación: {df[Estado_envio].isna().sum()}")
print(f"\nDistribución final de Estado_Envio:")
print(df[Estado_envio].value_counts())

Valor más frecuente (moda) de Estado_Envio: Retrasado

Nulos antes de imputación: 1683
Nulos después de imputación: 0

Distribución final de Estado_Envio:
Estado_Envio
Retrasado    3440
Entregado    1684
Devuelto     1645
En Camino    1628
Perdido      1603
Name: count, dtype: int64


In [96]:
df['Cantidad_Vendida'].describe()

count    10000.000000
mean         7.513600
std          4.024082
min          1.000000
25%          4.000000
50%          7.000000
75%         11.000000
max         14.000000
Name: Cantidad_Vendida, dtype: float64

In [97]:
df.to_csv('data/transaccion_logistica_limpio.csv', index=False)

In [98]:
df_feedback_clientes_limpio = pd.read_csv('data/feedback_clientes_limpio.csv')

In [99]:
df_inventario_limpio = pd.read_csv('data/inventario_central_limpio.csv')

In [100]:
# Join entre df (transacciones) y df_feedback_clientes_limpio por Transaccion_ID
df_merged = pd.merge(df, df_feedback_clientes_limpio, on='Transaccion_ID', how='inner')

print(f"Filas en df: {len(df)}")
print(f"Filas en df_feedback_clientes_limpio: {len(df_feedback_clientes_limpio)}")
print(f"Filas en df_merged: {len(df_merged)}")
print(f"\nColumnas en df_merged:\n{df_merged.columns.tolist()}")
print(f"\nPrimeras filas del resultado:")
print(df_merged.head())

Filas en df: 10000
Filas en df_feedback_clientes_limpio: 4500
Filas en df_merged: 4500

Columnas en df_merged:
['Transaccion_ID', 'SKU_ID', 'Fecha_Venta', 'Cantidad_Vendida', 'Precio_Venta_Final', 'Costo_Envio', 'Tiempo_Entrega_Real', 'Estado_Envio', 'Ciudad_Destino', 'Canal_Venta', 'Feedback_ID', 'Rating_Producto', 'Rating_Logistica', 'Comentario_Texto', 'Recomienda_Marca', 'Ticket_Soporte_Abierto', 'Edad_Cliente', 'Satisfaccion_NPS']

Primeras filas del resultado:
  Transaccion_ID     SKU_ID Fecha_Venta  Cantidad_Vendida  Precio_Venta_Final  \
0      TRX-10000  PROD-3619  2025-04-25                 5             1752.48   
1      TRX-10001  PROD-1456  2025-07-29                12              686.88   
2      TRX-10008  PROD-3771  2025-11-03                 1             1404.98   
3      TRX-10008  PROD-3771  2025-11-03                 1             1404.98   
4      TRX-10010  PROD-1356  2025-04-04                 7              917.18   

   Costo_Envio  Tiempo_Entrega_Real Estado

In [101]:
df_merged.describe()

,Fecha_Venta,Cantidad_Vendida,Precio_Venta_Final,Costo_Envio,Tiempo_Entrega_Real,Rating_Producto,Rating_Logistica,Edad_Cliente,Satisfaccion_NPS
count,4500,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000
mean,2025-05-30 13:52:38.400000,7.605556,993.638922,52.655562,15.022000,2.995778,3.005111,50.431778,0.303089
min,2024-09-23 00:00:00,1.000000,10.010000,5.060000,1.000000,1.000000,1.000000,18.000000,-99.800000
25%,2025-01-27 18:00:00,4.000000,498.710000,31.512500,8.000000,2.000000,2.000000,34.000000,-49.200000
50%,2025-05-29 00:00:00,8.000000,992.560000,52.360000,15.000000,3.000000,3.000000,50.000000,1.100000
75%,2025-10-02 00:00:00,11.000000,1496.225000,74.095000,22.000000,4.000000,4.000000,67.000000,49.525000
max,2026-02-04 00:00:00,14.000000,1999.580000,99.930000,29.000000,5.000000,5.000000,84.000000,99.900000
std,NaN,4.088835,575.208530,26.052381,8.382032,1.424465,1.418520,19.242859,57.184315


In [102]:
df_merged.columns

Index(['Transaccion_ID', 'SKU_ID', 'Fecha_Venta', 'Cantidad_Vendida',
       'Precio_Venta_Final', 'Costo_Envio', 'Tiempo_Entrega_Real',
       'Estado_Envio', 'Ciudad_Destino', 'Canal_Venta', 'Feedback_ID',
       'Rating_Producto', 'Rating_Logistica', 'Comentario_Texto',
       'Recomienda_Marca', 'Ticket_Soporte_Abierto', 'Edad_Cliente',
       'Satisfaccion_NPS'],
      dtype='object')

In [103]:
# Join entre df_merged (transacciones + feedback) y df_inventario_limpio por SKU_ID
df_merged = pd.merge(df_merged, df_inventario_limpio, on='SKU_ID', how='inner')

print(f"Filas después del merge con inventario: {len(df_merged)}")
print(f"\nColumnas en df_merged:\n{df_merged.columns.tolist()}")
print(f"\nPrimeras filas del resultado:")
print(df_merged.head())

Filas después del merge con inventario: 3719

Columnas en df_merged:
['Transaccion_ID', 'SKU_ID', 'Fecha_Venta', 'Cantidad_Vendida', 'Precio_Venta_Final', 'Costo_Envio', 'Tiempo_Entrega_Real', 'Estado_Envio', 'Ciudad_Destino', 'Canal_Venta', 'Feedback_ID', 'Rating_Producto', 'Rating_Logistica', 'Comentario_Texto', 'Recomienda_Marca', 'Ticket_Soporte_Abierto', 'Edad_Cliente', 'Satisfaccion_NPS', 'Categoria', 'Stock_Actual', 'Costo_Unitario_USD', 'Punto_Reorden', 'Lead_Time_Dias', 'Bodega_Origen', 'Ultima_Revision']

Primeras filas del resultado:
  Transaccion_ID     SKU_ID Fecha_Venta  Cantidad_Vendida  Precio_Venta_Final  \
0      TRX-10001  PROD-1456  2025-07-29                12              686.88   
1      TRX-10010  PROD-1356  2025-04-04                 7              917.18   
2      TRX-10016  PROD-1895  2025-03-04                 3              475.85   
3      TRX-10016  PROD-1895  2025-03-04                 3              475.85   
4      TRX-10019  PROD-3465  2025-09-04     

In [104]:
df_merged.to_csv('data/transaccion_completa_limpio.csv', index=False)

In [105]:
df_merged['Rating_Servicio'] = (df_merged['Rating_Producto'] * df_merged['Rating_Logistica'])/5

In [106]:
df_merged.to_csv('data/transaccion_completa_limpio.csv', index=False)